## **Mounting the Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **Pyserini Section**

In [ ]:
!pip install pyserini==0.9.4.0

     |████████████████████████████████| 60.4MB 67kB/s 
     |████████████████████████████████| 1.1MB 45.3MB/s 


In [ ]:
%cd /content/drive/My Drive/QASC-DATASET/data/QASC_Corpus/anserini

/content/drive/My Drive/QASC-DATASET/data/QASC_Corpus/anserini


In [ ]:
import os       #importing os to set environment variable
def install_java():
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null      #install openjdk
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"     #set environment variable
  !java -version     #check java version
install_java()

openjdk version "11.0.8" 2020-07-14
OpenJDK Runtime Environment (build 11.0.8+10-post-Ubuntu-0ubuntu118.04.1)
OpenJDK 64-Bit Server VM (build 11.0.8+10-post-Ubuntu-0ubuntu118.04.1, mixed mode, sharing)


# **Clone the Pyserini Repository**

In [ ]:
!git clone https://github.com/castorini/pyserini.git

Cloning into 'pyserini'...
remote: Enumerating objects: 1267, done.
remote: Total 1267 (delta 0), reused 0 (delta 0), pack-reused 1267
Receiving objects: 100% (1267/1267), 378.78 KiB | 2.85 MiB/s, done.
Resolving deltas: 100% (810/810), done.


In [ ]:
#%cd pyserini
#!git clone https://github.com/castorini/anserini-tools.git
%cd /content/drive/My Drive/QASC-DATASET/data/QASC_Corpus/anserini

/content/drive/My Drive/QASC-DATASET/data/QASC_Corpus/anserini


In [ ]:
sentences= []
index=[]
i=1
corpus_sentences=open('/content/drive/My Drive/QASC-DATASET/data/QASC_Corpus/QASC_Corpus.txt','r')
for line in corpus_sentences.readlines():
    lin = line.replace('\n', ' ')
    index.append(i)
    sentences.append(lin)
    i=i+1
  

In [ ]:
print(index[100000])


100001


In [ ]:
import pandas as pd
df = pd.DataFrame(list(zip(index, sentences)), columns=['index','paragraph'])
df.to_csv('/content/drive/My Drive/QASC-DATASET/data/QASC_Corpus/anserini/pyserini/collections/document_passages.tsv', header=False, index=False, sep='\t')

**Loading input Corpus dataset in TSV format and converting it into JSONL with each file containing max 1000000(1M) Sentences**

In [ ]:
df_data= pd.read_csv('/content/drive/My Drive/QASC-DATASET/data/QASC_Corpus/anserini/pyserini/collections/document_passages.tsv', sep='\t')

In [ ]:
!python pyserini/anserini-tools/scripts/msmarco/convert_collection_to_jsonl.py \
 --collection-path pyserini/collections/document_passages.tsv \
 --output-folder pyserini/collections/collection_jsonl

Converting collection...
Converted 0 docs, writing into file 1
Converted 100,000 docs, writing into file 1
Converted 200,000 docs, writing into file 1
Converted 300,000 docs, writing into file 1
Converted 400,000 docs, writing into file 1
Converted 500,000 docs, writing into file 1
Converted 600,000 docs, writing into file 1
Converted 700,000 docs, writing into file 1
Converted 800,000 docs, writing into file 1
Converted 900,000 docs, writing into file 1
Converted 1,000,000 docs, writing into file 2
Converted 1,100,000 docs, writing into file 2
Converted 1,200,000 docs, writing into file 2
Converted 1,300,000 docs, writing into file 2
Converted 1,400,000 docs, writing into file 2
Converted 1,500,000 docs, writing into file 2
Converted 1,600,000 docs, writing into file 2
Converted 1,700,000 docs, writing into file 2
Converted 1,800,000 docs, writing into file 2
Converted 1,900,000 docs, writing into file 2
Converted 2,000,000 docs, writing into file 3
Converted 2,100,000 docs, writing i

**Creating JSON collections for the corpus**

In [ ]:
!python -m pyserini.index -collection JsonCollection -generator DefaultLuceneDocumentGenerator \
 -threads 9 -input pyserini/collections/collection_jsonl \
 -index pyserini/indexes/lucene-index-qasc -storePositions -storeDocvectors -storeRaw

2020-10-21 03:32:33,608 INFO  [main] index.IndexCollection (IndexCollection.java:636) - Setting log level to INFO
2020-10-21 03:32:33,611 INFO  [main] index.IndexCollection (IndexCollection.java:639) - Starting indexer...
2020-10-21 03:32:33,611 INFO  [main] index.IndexCollection (IndexCollection.java:640) - ============ Loading Parameters ============
2020-10-21 03:32:33,612 INFO  [main] index.IndexCollection (IndexCollection.java:641) - DocumentCollection path: pyserini/collections/collection_jsonl
2020-10-21 03:32:33,612 INFO  [main] index.IndexCollection (IndexCollection.java:642) - CollectionClass: JsonCollection
2020-10-21 03:32:33,613 INFO  [main] index.IndexCollection (IndexCollection.java:643) - Generator: DefaultLuceneDocumentGenerator
2020-10-21 03:32:33,613 INFO  [main] index.IndexCollection (IndexCollection.java:644) - Threads: 9
2020-10-21 03:32:33,614 INFO  [main] index.IndexCollection (IndexCollection.java:645) - Stemmer: porter
2020-10-21 03:32:33,614 INFO  [main] inde

**Sample Testing of the indexing done by the Pyserini**

In [ ]:
from pyserini.search import SimpleSearcher
import pandas as pd
from os.path import join
import numpy as np

searcher = SimpleSearcher('pyserini/indexes/lucene-index-qasc')

searcher.set_bm25(1.2, 0.75)

hits = searcher.search("The dog is sleeping")

# Print the first 10 hits:
for hit in hits:
    print(f'{hit.docid:15} {hit.score:.5f}')

1339157         8.06910
3656512         8.06910
4791219         8.06910
12316289        7.82960
3322835         7.68200
1283510         7.53660
8128791         7.53660
8479497         7.53660
9651509         7.53660
13082459        7.07000


In [ ]:
d=searcher.doc(str(hits[0].docid))
print(d.raw)

<bound method Document.raw of <pyserini.search._base.Document object at 0x7f7b675a0278>>


In [ ]:
print(hits)

[<io.anserini.search.SimpleSearcher$Result at 0x7f7bf64b8518 jclass=io/anserini/search/SimpleSearcher$Result jself=<LocalRef obj=0x2ca0898 at 0x7f7c1acbeb50>>, <io.anserini.search.SimpleSearcher$Result at 0x7f7b67551518 jclass=io/anserini/search/SimpleSearcher$Result jself=<LocalRef obj=0x2ca0900 at 0x7f7c1acbe5f0>>, <io.anserini.search.SimpleSearcher$Result at 0x7f7b67551200 jclass=io/anserini/search/SimpleSearcher$Result jself=<LocalRef obj=0x2ca0908 at 0x7f7c1acbef50>>, <io.anserini.search.SimpleSearcher$Result at 0x7f7b67551468 jclass=io/anserini/search/SimpleSearcher$Result jself=<LocalRef obj=0x2ca0950 at 0x7f7c1acbec30>>, <io.anserini.search.SimpleSearcher$Result at 0x7f7b67551410 jclass=io/anserini/search/SimpleSearcher$Result jself=<LocalRef obj=0x2ca0968 at 0x7f7c1acbebf0>>, <io.anserini.search.SimpleSearcher$Result at 0x7f7b675513b8 jclass=io/anserini/search/SimpleSearcher$Result jself=<LocalRef obj=0x2ca0970 at 0x7f7c1acbe650>>, <io.anserini.search.SimpleSearcher$Result at 

In [ ]:
R=[]
doc = searcher.doc(str(hits[0].docid))
R.append(doc.raw())

In [ ]:
import json 

!pip install jsonlines
import pandas as pd

# **Getting the Input QASC DATASET to proper Format**

#Training set

In [ ]:
import jsonlines
id=[]  #done
question=[]   #done
label_list=[] #done
A=[] #done
B=[] #done
C=[] #done
D=[] #done
E=[] #done
F=[] #done
G=[] #done
H=[] #done
Actual_Answer=[] #done
fact_1=[] #done
fact_2=[] #done
fact_combined=[] #done
with jsonlines.open('/content/drive/My Drive/QASC-DATASET/data/QASC_Dataset/train.jsonl') as f:

    for line in f.iter():
      question.append(line['question']['stem'])
      label_list.append(line['question']['choices'])
      id.append(line["id"])
      Actual_Answer.append(line['answerKey'])
      fact_1.append(line['fact1'])
      fact_2.append(line['fact2'])
      fact_combined.append(line['combinedfact'])

#extracting label options into their list
for i in range(0,len(label_list)):
  for j in range(0,8):
    if(label_list[i][j]['label']=="A"):
      A.append(label_list[i][j]['text'])
      
    if(label_list[i][j]['label']=="B"):
      B.append(label_list[i][j]['text'])
      
    if(label_list[i][j]['label']=="C"):
      C.append(label_list[i][j]['text'])
      
    if(label_list[i][j]['label']=="D"):
      D.append(label_list[i][j]['text'])
      
    if(label_list[i][j]['label']=="E"):
      E.append(label_list[i][j]['text'])
      
    if(label_list[i][j]['label']=="F"):
      F.append(label_list[i][j]['text'])
      
    if(label_list[i][j]['label']=="G"):
      G.append(label_list[i][j]['text'])
      
    if(label_list[i][j]['label']=="H"):
      H.append(label_list[i][j]['text'])
for i in range(0,len(Actual_Answer)):
  if(Actual_Answer[i]=="A"):
    Actual_Answer[i]=A[i] 
  if(Actual_Answer[i]=="B"):
    Actual_Answer[i]=B[i] 
  if(Actual_Answer[i]=="C"):
    Actual_Answer[i]=C[i] 
  if(Actual_Answer[i]=="D"):
    Actual_Answer[i]=D[i] 
  if(Actual_Answer[i]=="E"):
    Actual_Answer[i]=E[i] 
  if(Actual_Answer[i]=="F"):
    Actual_Answer[i]=F[i] 
  if(Actual_Answer[i]=="G"):
    Actual_Answer[i]=G[i] 
  if(Actual_Answer[i]=="H"):
    Actual_Answer[i]=H[i]  
      
  
merged_list = tuple(zip(id,question,Actual_Answer,A,B,C,D,E,F,G,H,fact_1,fact_2,fact_combined))
training_data=pd.DataFrame(merged_list,columns=['ID','Question','Actual Answer','A','B','C','D','E','F','G','H','Fact 1','Fact 2','Fact Combined'])
#pd.read_json (r'/content/drive/My Drive/QASC-DATASET/data/QASC_Dataset/train.jsonl')

##############################################################################
answer_queries=[]
for i in range(0,len(A)):
    answer_queries.append(A[i])
    answer_queries.append(B[i])
    answer_queries.append(C[i])
    answer_queries.append(D[i])
    answer_queries.append(E[i])
    answer_queries.append(F[i])
    answer_queries.append(G[i])
    answer_queries.append(H[i])



**Extracting Sentences from corpus for each answer candidate**

In [ ]:
###########################################################################
from pyserini.search import SimpleSearcher
import pandas as pd
from os.path import join
import numpy as np

searcher = SimpleSearcher('pyserini/indexes/lucene-index-qasc')

searcher.set_bm25(1.2, 0.75)
answer_candidates=[]
for i in range(0,len(answer_queries)):
  # print(i)
  hits = searcher.search(answer_queries[i])
  if(len(hits)!=0):
    doc = searcher.doc(str(hits[0].docid))
    answer_candidates.append(doc.raw())
  else:
    answer_candidates.append("")
  # print(answer_candidates[i])
############################################################################
question_actaual_answer_dataframe=[]
answer_candidates_dataframe=[]
for i in range(0,len(question)):
  answer_candidates_dataframe.append(fact_1[i])
  answer_candidates_dataframe.append(fact_2[i])
  answer_candidates_dataframe.append(fact_combined[i])
  question_actaual_answer_dataframe.append(question[i]+" "+Actual_Answer[i])
  question_actaual_answer_dataframe.append(question[i]+" "+Actual_Answer[i])
  question_actaual_answer_dataframe.append(question[i]+" "+Actual_Answer[i])
  for j in range(i*8,i*8+8):
    answer_candidates_dataframe.append(answer_candidates[j])
    question_actaual_answer_dataframe.append(question[i]+" "+Actual_Answer[i])
###############################################################################
QASC_df=pd.DataFrame({'Query':question_actaual_answer_dataframe,'Sentence':answer_candidates_dataframe})



In [ ]:
##############################################################################
QASC_df.to_csv('/content/drive/My Drive/QASC-DATASET/data/QASC_Dataset/train_df.csv', index=False)

## DEV Set

In [ ]:
import jsonlines
id=[]  #done
question=[]   #done
label_list=[] #done
A=[] #done
B=[] #done
C=[] #done
D=[] #done
E=[] #done
F=[] #done
G=[] #done
H=[] #done
Actual_Answer=[] #done
fact_1=[] #done
fact_2=[] #done
fact_combined=[] #done
with jsonlines.open('/content/drive/My Drive/QASC-DATASET/data/QASC_Dataset/dev.jsonl') as f:

    for line in f.iter():
      question.append(line['question']['stem'])
      label_list.append(line['question']['choices'])
      id.append(line["id"])
      Actual_Answer.append(line['answerKey'])
      fact_1.append(line['fact1'])
      fact_2.append(line['fact2'])
      fact_combined.append(line['combinedfact'])

#extracting label options into their list
for i in range(0,len(label_list)):
  for j in range(0,8):
    if(label_list[i][j]['label']=="A"):
      A.append(label_list[i][j]['text'])
      
    if(label_list[i][j]['label']=="B"):
      B.append(label_list[i][j]['text'])
      
    if(label_list[i][j]['label']=="C"):
      C.append(label_list[i][j]['text'])
      
    if(label_list[i][j]['label']=="D"):
      D.append(label_list[i][j]['text'])
      
    if(label_list[i][j]['label']=="E"):
      E.append(label_list[i][j]['text'])
      
    if(label_list[i][j]['label']=="F"):
      F.append(label_list[i][j]['text'])
      
    if(label_list[i][j]['label']=="G"):
      G.append(label_list[i][j]['text'])
      
    if(label_list[i][j]['label']=="H"):
      H.append(label_list[i][j]['text'])
for i in range(0,len(Actual_Answer)):
  if(Actual_Answer[i]=="A"):
    Actual_Answer[i]=A[i] 
  if(Actual_Answer[i]=="B"):
    Actual_Answer[i]=B[i] 
  if(Actual_Answer[i]=="C"):
    Actual_Answer[i]=C[i] 
  if(Actual_Answer[i]=="D"):
    Actual_Answer[i]=D[i] 
  if(Actual_Answer[i]=="E"):
    Actual_Answer[i]=E[i] 
  if(Actual_Answer[i]=="F"):
    Actual_Answer[i]=F[i] 
  if(Actual_Answer[i]=="G"):
    Actual_Answer[i]=G[i] 
  if(Actual_Answer[i]=="H"):
    Actual_Answer[i]=H[i]  
      
  
merged_list = tuple(zip(id,question,Actual_Answer,A,B,C,D,E,F,G,H,fact_1,fact_2,fact_combined))
training_data=pd.DataFrame(merged_list,columns=['ID','Question','Actual Answer','A','B','C','D','E','F','G','H','Fact 1','Fact 2','Fact Combined'])
#pd.read_json (r'/content/drive/My Drive/QASC-DATASET/data/QASC_Dataset/train.jsonl')

##############################################################################
answer_queries=[]
for i in range(0,len(A)):
    answer_queries.append(A[i])
    answer_queries.append(B[i])
    answer_queries.append(C[i])
    answer_queries.append(D[i])
    answer_queries.append(E[i])
    answer_queries.append(F[i])
    answer_queries.append(G[i])
    answer_queries.append(H[i])


# Extracting Sentences from corpus for each answer candidate

In [ ]:
###########################################################################
from pyserini.search import SimpleSearcher
import pandas as pd
from os.path import join
import numpy as np

searcher = SimpleSearcher('pyserini/indexes/lucene-index-qasc')

searcher.set_bm25(1.2, 0.75)
answer_candidates=[]
for i in range(0,len(answer_queries)):
  # print(i)
  hits = searcher.search(answer_queries[i])
  if(len(hits)!=0):
    doc = searcher.doc(str(hits[0].docid))
    answer_candidates.append(doc.raw())
  else:
    answer_candidates.append("")
  # print(answer_candidates[i])
############################################################################
question_actaual_answer_dataframe=[]
answer_candidates_dataframe=[]
for i in range(0,len(question)):
  answer_candidates_dataframe.append(fact_1[i])
  answer_candidates_dataframe.append(fact_2[i])
  answer_candidates_dataframe.append(fact_combined[i])
  question_actaual_answer_dataframe.append(question[i]+" "+fact_combined[i])
  question_actaual_answer_dataframe.append(question[i]+" "+Actual_Answer[i])
  question_actaual_answer_dataframe.append(question[i]+" "+Actual_Answer[i][i])
  for j in range(i*8,i*8+8):
    answer_candidates_dataframe.append(answer_candidates[j])
    question_actaual_answer_dataframe.append(fact_combined[i])
###############################################################################
QASC_df=pd.DataFrame({'Query':question_actaual_answer_dataframe,'Sentence':answer_candidates_dataframe})




# Exporting the dataframe

In [ ]:
##############################################################################
QASC_df.to_csv('/content/drive/My Drive/QASC-DATASET/data/QASC_Dataset/dev_df_factc.csv', index=False)